# Set up

In [1]:
!pip3 install face_recognition

    100% |████████████████████████████████| 100.2MB 336kB/s 
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!ls '/content/drive/Team Drives/Senior Project/Docker/newData/'

aoi   bow  fang  lin	mint   nam  ning  patt	whan  ying
auey  eye  juti  meiji	mintg  nat  o	  sand	yeen  yuan


In [0]:
data = '/content/drive/Team Drives/Senior Project/Docker/newData/'

In [0]:
import pandas as pd
import pickle
import numpy as np
import math
from sklearn import neighbors
import os
import os.path
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
from google.colab import files

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

# Prepare Data

In [6]:
X = []
y = []

count_allimg = 0 
count_failimg = 0 

# Loop through each person in the training set
for class_dir in os.listdir(data):
    if not os.path.isdir(os.path.join(data, class_dir)):
        continue

    # Loop through each training image for the current person
    for img_path in image_files_in_folder(os.path.join(data, class_dir)):
        image = face_recognition.load_image_file(img_path)
        face_bounding_boxes = face_recognition.face_locations(image)
        
        count_allimg += 1

        if len(face_bounding_boxes) != 1:
            # If there are no people (or too many people) in a training image, skip the image.
            count_failimg += 1
            print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
        else:
            # Add face encoding for current image to the training set
            X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
            y.append(class_dir)
            
print("count all image : ", count_allimg)
print("count fail image : ", count_failimg)

Image /content/drive/Team Drives/Senior Project/Docker/newData/lin/frame643.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame44.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame40.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame41.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame43.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame42.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame57.jpg not suitable for training: Didn't find a face
Image /content/drive/Team Drives/Senior Project/Docker/newData/bow/frame55.jpg not suitable for training: Didn't find a face

# Train Test

In [7]:
# Split our data
train, test, train_labels, test_labels = train_test_split(X,
                                                          y,
                                                          test_size=0.3,
                                                          random_state=42)

# Create and train the KNN classifier
# n_neighbors=2, algorithm='ball_tree', weights='distance'
knn_clf = neighbors.KNeighborsClassifier()
knn_clf.fit(train, train_labels)
knn_preds = knn_clf.predict(test)

gnb = GaussianNB()
gnb_clf = gnb.fit(train, train_labels)
gnb_preds = gnb_clf.predict(test)

dtc_clf = tree.DecisionTreeClassifier()
dtc_clf = dtc_clf.fit(train, train_labels)
dtc_prediction = dtc_clf.predict(test)

rfc_clf = RandomForestClassifier()
rfc_clf.fit(train, train_labels)
rfc_prediction = rfc_clf.predict(test)


# # Evaluate accuracy
print("KNN : " , accuracy_score(test_labels, knn_preds))
print("gnb : " , accuracy_score(test_labels, gnb_preds))
print("DecisionTreeClassifier : " , accuracy_score(test_labels, dtc_prediction))
print("RandomForestClassifier : " , accuracy_score(test_labels, rfc_prediction))

KNN :  1.0
gnb :  0.9910714285714286
DecisionTreeClassifier :  0.8392857142857143
RandomForestClassifier :  0.9732142857142857


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
